In [1]:
#Need to compare the BERT model with the regular SVM model 

In [10]:
from datasets import load_dataset
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import pickle
import os
from sklearn.metrics import classification_report, accuracy_score, f1_score, plot_confusion_matrix,ConfusionMatrixDisplay, confusion_matrix
import pickle
os.environ["WANDB_DISABLED"] = "true"

In [14]:
model_name = "../FinalBERT/Models/Final_model/BERT_13classes_final"
max_length=128

In [15]:
#loading the label encoder
le = pickle.load(open("../FinalBERT/Data/label_encoder.pkl", "rb"))

In [16]:
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

In [17]:
target_names=le.transform(le.classes_)

In [18]:
target_names

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [23]:
""" Parameters to test"""    
params_combos= {
	  "per_gpu_batch_size": [8, 16, 32],
	  "learning_rate": [2e-5, 3e-5, 5e-5],
	  "num_epochs": [20, 30, 40]
      #"max_length": [32, 128, 256, 512]
	}

    

In [20]:
def preprocess_function(examples):
    #print(examples['TEXT'])
    return tokenizer(examples["text"], padding=True, truncation=True)

    """Uncomment the following to have max length set"""
    #return tokenizer(examples["text"],padding=True,truncation=True, max_length=max_length)
    
    """Uncomment the following to have max length change from the params"""
    #return to tokenizer(examples['text'], padding=True, truncation=True, max_length=params['max_length'][0])

In [24]:
#traying 5 combinations first
import random
params_combos = []
i = 0
while i<10:
    first = random.randint(0,2)
    second = random.randint(0,2)
    third = random.randint(0,2)
    param = {"batch_size": params["per_gpu_batch_size"][first],
             "lr": params["learning_rate"][second],
             "epochs": params["num_epochs"][third]}
    if param not in params_combos:
        params_combos.append(param)
        i+=1

In [25]:
params_combos

[{'batch_size': 32, 'lr': 5e-05, 'epochs': 40},
 {'batch_size': 8, 'lr': 3e-05, 'epochs': 40},
 {'batch_size': 32, 'lr': 3e-05, 'epochs': 30},
 {'batch_size': 16, 'lr': 3e-05, 'epochs': 40},
 {'batch_size': 16, 'lr': 5e-05, 'epochs': 30},
 {'batch_size': 16, 'lr': 3e-05, 'epochs': 30},
 {'batch_size': 16, 'lr': 5e-05, 'epochs': 20},
 {'batch_size': 32, 'lr': 5e-05, 'epochs': 20},
 {'batch_size': 32, 'lr': 5e-05, 'epochs': 30},
 {'batch_size': 8, 'lr': 2e-05, 'epochs': 20}]

In [27]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, 
                       truncation=True, 
                       #max_length=max_length,
                       return_tensors="pt").to("cuda")
    
    # perform inference to our model
    outputs = model(**inputs)
    #print(outputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    #print(outputs[0].softmax(1).argmax())
    # executing argmax function to get the candidate label
    return probs.argmax().item()

## Kfold

In [150]:
#Run kfold over here 
"""
results = []
f1_scores = []
best_model =[]
for i in range(5):
    train_dataset = load_dataset('csv', data_files={'train': 'Data/train.csv'})
    #train_dataset = load_dataset('csv', data_files={'train': 'Data/train2.csv'})
    testing_dataset =load_dataset("csv", data_files={'test':'Data/test.csv'}) 
    #eval_dataset =load_dataset("csv", data_files={'test':'Data/eval.csv'}) 

    #print(train_dataset["train"][0])

    tokenized_dataset_train =train_dataset.map(preprocess_function)
    #print(tokenized_dataset_train['train']['input_ids'])

    tokenized_dataset_test = testing_dataset.map(preprocess_function)
    
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    #dataset has been loaded correctly. 
    print("----------------------MODEL "+ str(i)+"--------------------------------")

    for p in params_combos:
        
        print("Combination: ", p)
        
        model=BertForSequenceClassification.from_pretrained(model_name, num_labels=13)
        #device="cpu"
        device="cuda"
        model.to(device)


        training_args = TrainingArguments(
        num_train_epochs=p["epochs"],
        learning_rate=p["lr"],
        per_device_train_batch_size=p["batch_size"],  # batch size per device during training
        weight_decay=0.01,               # strength of weight decay
        #oad_best_model_at_end=True,
        logging_steps=100,
        #valuation_strategy="steps",
        output_dir="Models/{NAME_OF_OUTPUT_FILE}" #Name of outputs to store checkpoints 
        )

        trainer = Trainer( model=model,
                      args=training_args,
                      train_dataset=tokenized_dataset_train['train'],
                      #eval_dataset=tokenized_dataset_valid['valid'],
                      tokenizer=tokenizer,
                      data_collator=data_collator
                     )


        trainer.train()

        y_pred=[]

        for text in tokenized_dataset_test["test"]["text"]:
            y_pred.append(get_prediction(text))

        predictions = le.inverse_transform(y_pred)
        y_true = tokenized_dataset_test['test']["label"]
        
        f1= f1_score(le.inverse_transform(y_true), predictions, average="macro")
        f1_scores.append(f1)
        
        #Saving the best model 
        if f1 > max(f1_scores):
            best_model = {'split': i, 
                          'params': p,
                          'f1_score': f1,
                          'accuracy': acc}

        acc = accuracy_score(le.inverse_transform(y_true), predictions)
        results.append(acc)
        print(classification_report(le.inverse_transform(y_true), predictions))
        
"""

Using custom data configuration default-39460bdd07acc7bf
Reusing dataset csv (C:\Users\adianidm\.cache\huggingface\datasets\csv\default-39460bdd07acc7bf\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-38d3ac2bff9f4eb2
Reusing dataset csv (C:\Users\adianidm\.cache\huggingface\datasets\csv\default-38d3ac2bff9f4eb2\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\adianidm\.cache\huggingface\datasets\csv\default-39460bdd07acc7bf\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-0c03d44608866328.arrow
Loading cached processed dataset at C:\Users\adianidm\.cache\huggingface\datasets\csv\default-38d3ac2bff9f4eb2\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-96dbeb40b209ba5f.arrow
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\adianidm/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
 

...

              precision    recall  f1-score   support

         ack       1.00      1.00      1.00         3
         dno       1.00      0.80      0.89        10
          fa       1.00      1.00      1.00         9
          fe       0.82      0.90      0.86        10
          fp       0.93      0.88      0.90        16
          ft       0.90      1.00      0.95         9
           n       1.00      0.70      0.82        10
         n-d       0.67      1.00      0.80         2
       query       1.00      0.75      0.86         8
         sno       0.84      0.93      0.88        69
          so       0.50      0.38      0.43         8
           y       0.86      0.60      0.71        10
         y-d       0.25      1.00      0.40         1

    accuracy                           0.85       165
   macro avg       0.83      0.84      0.81       165
weighted avg       0.87      0.85      0.85       165

Combination:  {'batch_size': 16, 'lr': 2e-05, 'epochs': 30}


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\adianidm/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
   

Step,Training Loss
100,1.682700
200,0.737600
300,0.295800
400,0.122300
500,0.055800
600,0.034100
700,0.030700
800,0.024000
900,0.021800
1000,0.022700


Saving model checkpoint to Models/BERT_13classes\checkpoint-500
Configuration saved in Models/BERT_13classes\checkpoint-500\config.json
Model weights saved in Models/BERT_13classes\checkpoint-500\pytorch_model.bin
tokenizer config file saved in Models/BERT_13classes\checkpoint-500\tokenizer_config.json
Special tokens file saved in Models/BERT_13classes\checkpoint-500\special_tokens_map.json
Saving model checkpoint to Models/BERT_13classes\checkpoint-1000
Configuration saved in Models/BERT_13classes\checkpoint-1000\config.json
Model weights saved in Models/BERT_13classes\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in Models/BERT_13classes\checkpoint-1000\tokenizer_config.json
Special tokens file saved in Models/BERT_13classes\checkpoint-1000\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




              precision    recall  f1-score   support

         ack       0.75      1.00      0.86         3
         dno       1.00      0.80      0.89        10
          fa       1.00      1.00      1.00         9
          fe       0.83      1.00      0.91        10
          fp       0.88      0.88      0.88        16
          ft       1.00      1.00      1.00         9
           n       0.80      0.80      0.80        10
         n-d       1.00      0.50      0.67         2
       query       1.00      0.62      0.77         8
         sno       0.83      0.93      0.88        69
          so       0.33      0.12      0.18         8
           y       0.86      0.60      0.71        10
         y-d       0.25      1.00      0.40         1

    accuracy                           0.84       165
   macro avg       0.81      0.79      0.76       165
weighted avg       0.84      0.84      0.83       165



In [18]:
# Best model parameters-Comment out if using KFold
best params ={'epochs': 40, 
              'lr': 5e-5,
              'batch_size': 8}
            

In [ ]:
#Comment out when not using KFold
#best_params = best_model['p']
#best_split = best_model['split']

## Train best estimator for classification report and confusion matrix

In [ ]:
#Run kfold over here 
results = []
f1_scores=[]
best_model = []
i = 2 #Choose the split of data or just use train/test
train_dataset = load_dataset('csv', data_files={'train': 'Data/train.csv'})
#train_dataset = load_dataset('csv', data_files={'train': 'Data/train2.csv'})
testing_dataset =load_dataset("csv", data_files={'test':'Data/test.csv'}) 
#eval_dataset =load_dataset("csv", data_files={'test':'Data/eval.csv'}) 
#print(train_dataset["train"][0])

tokenized_dataset_train =train_dataset.map(preprocess_function)
#print(tokenized_dataset_train['train']['input_ids'])

tokenized_dataset_test = testing_dataset.map(preprocess_function)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
#dataset has been loaded correctly. 
print("----------------------MODEL " + str(i)+"-------------------------------")


model=BertForSequenceClassification.from_pretrained(model_name, num_labels=13)
#device="cpu"
device="cuda"
model.to(device)


training_args = TrainingArguments(
num_train_epochs=best_params["epochs"],
learning_rate=best_params["lr"],
per_device_train_batch_size=best_params["batch_size"],  # batch size per device during training
weight_decay=0.01,               # strength of weight decay
#oad_best_model_at_end=True,
logging_steps=100,
#valuation_strategy="steps",
output_dir="Models/BERT_13classes_output"
)

trainer = Trainer( model=model,
              args=training_args,
              train_dataset=tokenized_dataset_train['train'],
              #eval_dataset=tokenized_dataset_valid['valid'],
              tokenizer=tokenizer,
              data_collator=data_collator
             )


trainer.train()

y_pred=[]

for text in tokenized_dataset_test["test"]["text"]:
    y_pred.append(get_prediction(text))

predictions = le.inverse_transform(y_pred)
y_true = tokenized_dataset_test['test']["label"]

acc = accuracy_score(le.inverse_transform(y_true), predictions)
#f1_score = f1_score(le.inverse_transform(y_true), predictions, average="macro")
#f1_scores.append(f1_score)
results.append(acc)
print(classification_report(le.inverse_transform(y_true), predictions))
f1= f1_score(le.inverse_transform(y_true), predictions, average="macro")
f1_scores.append(f1)
cm = confusion_matrix(le.inverse_transform(y_true), predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
disp.plot()

trainer.save_model("Models/{NAME_OF_OUTPUT_FOLDER_FOR_FINAL_MODEL}")